In [1]:
%matplotlib inline

# Calculate electrochemical equilbria

In [12]:
from scipy import optimize

import fipy as fp

In [8]:
Vs = fp.Variable("0.018 l/mol")
R = fp.Variable("1 Nav * kB")
T = fp.Variable("298 K")
Faraday = fp.Variable("1 Nav * e")

simple mesh that only has a cell for electrode and a cell for the electrolyte

In [7]:
mesh = fp.Grid1D(nx=2)

In [30]:
def solid(X, interstitials, substitutionals, solvent):
    """solve for electron and cation concentration, given impurity anion and solvent
    """
    components = interstitials + substitutionals + [solvent]

    subs = 1. - X[0] - X[1]
    for Xj in components:
        subs -= Xj[0].value
        
    charge = 0. + X[0] * interstitials[0].z + X[1] * substitutionals[0].z
    for Xj in components:
        charge += Xj.z * Xj[0].value

    return [subs, charge]

In [71]:
def liquid(X, interstitials, substitutionals, solvent):
    """solve for solvent and anion concentration, given cation and impurity electron
    """
    components = interstitials + substitutionals + [solvent]

    subs = 1. - X[0] - X[1]
    for Xj in components:
        subs -= Xj[1].value
        
    charge = 0. + X[0] * solvent.z + X[1] * substitutionals[1].z
    for Xj in components:
        charge += Xj.z * Xj[1].value

    return [subs, charge]

## monovalent cation and salt

## $X_\mathrm{impurity} = 10^{-6}$

In [90]:
impurity = 1e-6

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+$", value=[0., (Vs * "1 mol/l").inBaseUnits()]),
    fp.CellVariable(mesh=mesh, name="$A^-$", value=[impurity, 0])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +1
substitutionals[1].z = -1
N.z                  = 0

components = interstitials + substitutionals + [N]

In [91]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj

-13.1223603774 $e^-$ [  4.99998500e-01   1.00000000e-06]
-3.32423534053 $M^+$ [ 0.4999995  0.018    ]
9.79807147978 $A^-$ [  1.00000000e-06   1.79990000e-02]
13.7788465736 $N$ [  1.00000000e-06   9.64000000e-01]


## $X_\mathrm{impurity} = 10^{-9}$

In [92]:
impurity = 1e-9

interstitials = [
    fp.CellVariable(mesh=mesh, name="$e^-$", value=[0., impurity])
]

substitutionals = [
    fp.CellVariable(mesh=mesh, name="$M^+$", value=[0., (Vs * "1 mol/l").inBaseUnits()]),
    fp.CellVariable(mesh=mesh, name="$A^-$", value=[impurity, 0])
]

N = fp.CellVariable(mesh=mesh, name="$N$", value=[impurity, 0.])

interstitials[0].z   = -1
substitutionals[0].z = +1
substitutionals[1].z = -1
N.z                  = 0

components = interstitials + substitutionals + [N]

In [93]:
interstitials[0][0] = 0.
substitutionals[0][0] = 0.
X0 = [1., .5]
X0 = optimize.fsolve(solid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

interstitials[0][0] = X0[0]
substitutionals[0][0] = X0[1]


N[1] = 0.
substitutionals[1][1] = 0.
X0 = [1., 1.]
X0 = optimize.fsolve(liquid, X0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N))

N[1] = X0[0]
substitutionals[1][1] = X0[1]

for Xj in components:
    Xj.muSL = fp.numerix.log(Xj[1] / Xj[0])
    print Xj.muSL, Xj.name, Xj

-20.0301186534 $e^-$ [  4.99999998e-01   1.00000000e-09]
-3.32423633953 $M^+$ [ 0.5    0.018]
16.7058822603 $A^-$ [  1.00000000e-09   1.79999990e-02]
20.6866018526 $N$ [  1.00000000e-09   9.64000000e-01]


In [ ]:
def equilibrium(Y, interstitials, substitutionals, solvent, YM):
    components = interstitials + substitutionals + [solvent]
    N = len(components)
    I = len(interstitials)
    
    YmL = 1.
    YmS = 1.
    for j, Yj in enumerate(interstitials):
        YmL += Y[j]
        YmS += Y[j + N]
        
    sumL = 1.
    sumS = 1.
    for j, Yj in enumerate(substitutionals + [solvent]):
        sumL -= Y[I + j]
        sumS -= Y[I + j + N]
    
    out = [Y[1] - YM]
    
    chargeL = 0.
    chargeS = 0.
    for j, Yj in enumerate(components):
        out.append(fp.numerix.exp(Yj.muSL + Yj.z * Y[2*N]) * Y[j + N] / YmS - Y[j] / YmL)
        chargeL += Yj.z * Y[j]
        chargeS += Yj.z * Y[j + N]
    
    out += [sumL, sumS, chargeL, chargeS]
    
    return out

components = interstitials + substitutionals + [N]

YM = (Vs * str(params["concentration"])).inBaseUnits()
Y0 = [3e-6, YM, YM, 1, 2, 1, 3e-6, 3e-6, 0]
Y0 = optimize.fsolve(equilibrium, Y0, xtol=1e-12, 
                     args=(interstitials, substitutionals, N, YM))